In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('../awesomebook-master/data/customer.csv')
  hotel_tb = pd.read_csv('../awesomebook-master/data/hotel.csv')
  reserve_tb = pd.read_csv('../awesomebook-master/data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('../awesomebook-master/data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('../awesomebook-master/data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('../awesomebook-master/data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('../awesomebook-master/data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

In [3]:
pd.merge(reserve_tb.query('people_num == 1'),
         hotel_tb.query('is_business'),
         on='hotel_id', how='inner')

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True
2,r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500,C,C-2,38.329097,140.698165,True
3,r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400,C,C-1,38.231842,140.797268,True
4,r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,13100,C,C-1,38.233985,140.795603,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,r3999,h_72,c_993,2016-06-09 04:35:48,2016-07-04,12:00:00,2016-07-07,1,29100,9700,B,B-2,35.545712,139.596157,True
442,r4001,h_253,c_993,2017-01-20 18:50:35,2017-02-10,11:00:00,2017-02-12,1,10400,5200,C,C-3,38.430220,140.696571,True
443,r4005,h_150,c_994,2016-03-25 11:51:26,2016-04-02,09:00:00,2016-04-03,1,25200,25200,B,B-2,35.546229,139.693687,True
444,r4011,h_137,c_995,2017-08-13 19:23:48,2017-09-07,09:30:00,2017-09-09,1,12400,6200,F,F-1,34.538252,132.469507,True


In [4]:
import gc

small_area_mst = hotel_tb.groupby(['big_area_name', 'small_area_name'], as_index=False) \
                         .size().reset_index()

print(small_area_mst)

small_area_mst.columns = ['Index', 'big_area_name', 'small_area_name', 'hotel_cnt']

small_area_mst['join_area_id'] = np.where(small_area_mst['hotel_cnt'] -1 >= 20,
                                          small_area_mst['small_area_name'],
                                          small_area_mst['big_area_name'])

small_area_mst.drop(['hotel_cnt', 'big_area_name'], axis=1, inplace=True)

base_hotel_mst = pd.merge(hotel_tb, small_area_mst, on='small_area_name') \
                   .loc[:, ['hotel_id', 'join_area_id']]

del small_area_mst
gc.collect()

base_hotel_mst

    index big_area_name small_area_name  size
0       0             A             A-1    35
1       1             A             A-3    30
2       2             B             B-1    15
3       3             B             B-2    18
4       4             B             B-3    19
5       5             C             C-1    25
6       6             C             C-2    30
7       7             C             C-3    21
8       8             D             D-1     7
9       9             D             D-2     7
10     10             D             D-3     7
11     11             D             D-4     5
12     12             E             E-1     8
13     13             E             E-2     5
14     14             E             E-3     4
15     15             E             E-4     9
16     16             F             F-1     6
17     17             F             F-2     5
18     18             F             F-3    11
19     19             F             F-4     7
20     20             G           

,hotel_id,join_area_id
0,h_1,D
1,h_2,A-1
2,h_3,E
3,h_4,C-3
4,h_5,G
...,...,...
295,h_296,F
296,h_297,A-3
297,h_298,A-1
298,h_299,C-2


In [5]:
recommend_hotel_mst = pd.concat([
    hotel_tb[['small_area_name', 'hotel_id']].rename(columns={'small_area_name': 'join_area_id'}, inplace=False),
    hotel_tb[['big_area_name', 'hotel_id']].rename(columns={'big_area_name': 'join_area_id'}, inplace=False)
])

recommend_hotel_mst.rename(columns={'hotel_id': 'rec_hotel_id'}, inplace=True)

pd.merge(base_hotel_mst, recommend_hotel_mst, on='join_area_id') \
  .loc[:, ['hotel_id', 'rec_hotel_id']] \
  .query('hotel_id != rec_hotel_id')

,hotel_id,rec_hotel_id
1,h_1,h_14
2,h_1,h_22
3,h_1,h_27
4,h_1,h_40
5,h_1,h_45
...,...,...
9658,h_300,h_247
9659,h_300,h_256
9660,h_300,h_257
9661,h_300,h_277


In [6]:
result = reserve_tb.groupby('customer_id') \
                   .apply(lambda group: group.sort_values(by='reserve_datetime', axis=0, inplace=False))

result['before_price'] = pd.Series(result['total_price'].shift(periods=2))

result

/var/folders/04/6s8yy3js1wj1c5n_56_fvj680000gn/T/ipykernel_7217/1697997927.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sort_values(by='reserve_datetime', axis=0, inplace=False))


reserve_id hotel_id customer_id     reserve_datetime  \
customer_id                                                             
c_1         0            r1     h_75         c_1  2016-03-06 13:09:42   
            1            r2    h_219         c_1  2016-07-16 23:39:55   
            2            r3    h_179         c_1  2016-09-24 10:03:17   
            3            r4    h_214         c_1  2017-03-08 03:20:10   
            4            r5     h_16         c_1  2017-09-05 19:50:37   
...                     ...      ...         ...                  ...   
c_999       4023      r4024    h_244       c_999  2016-10-06 18:01:34   
            4024      r4025    h_160       c_999  2017-03-11 11:56:05   
            4025      r4026    h_129       c_999  2017-06-27 23:00:02   
            4026      r4027     h_97       c_999  2017-09-29 05:24:57   
            4027      r4028     h_27       c_999  2018-03-14 05:01:45   

                 checkin_date checkin_time checkout_date  people_num  \
customer_id                                                            
c_1         0      2016-03-26     10:00:00    2016-03-29           4   
            1      2016-07-20     11:30:00    2016-07-21           2   
            2      2016-10-19     09:00:00    2016-10-22           2   
            3      2017-03-29     11:00:00    2017-03-30           4   
            4      2017-09-22     10:30:00    2017-09-23           3   
...                       ...          ...           ...         ...   
c_999       4023   2016-10-07     11:30:00    2016-10-09           3   
            4024   2017-03-27     10:00:00    2017-03-30           1   
            4025   2017-07-10     09:30:00    2017-07-11           2   
            4026   2017-10-09     10:30:00    2017-10-10           2   
            4027   2018-04-02     11:30:00    2018-04-04           2   

                  total_price  before_price  
customer_id                                  
c_1         0           97200           NaN  
            1           20600           NaN  
            2           33600       97200.0  
            3          194400       20600.0  
            4           68100       33600.0  
...                       ...           ...  
c_999       4023        59400       96000.0  
            4024        37200       62400.0  
            4025        16000       59400.0  
            4026        41800       37200.0  
            4027        74800       16000.0  

[4030 rows x 10 columns]

In [7]:
result = reserve_tb.groupby('customer_id') \
                   .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)) \
                   .reset_index(drop=True)

result['price_sum'] = pd.Series(
    result.loc[:, ["customer_id", "total_price"]]
          .groupby('customer_id')
          .rolling(center=False, window=3, min_periods=3).sum()
          .reset_index(drop=True)
          .loc[:, 'total_price']
)

result

/var/folders/04/6s8yy3js1wj1c5n_56_fvj680000gn/T/ipykernel_7217/975239264.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)) \


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100.0
...,...,...,...,...,...,...,...,...,...,...
4025,r4024,h_244,c_999,2016-10-06 18:01:34,2016-10-07,11:30:00,2016-10-09,3,59400,NaN
4026,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1,37200,159000.0
4027,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,112600.0
4028,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,95000.0


In [8]:
result = reserve_tb.groupby('customer_id') \
                   .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)) \
                   .reset_index(drop=True)

result['price_avg'] = pd.Series(
    result.groupby('customer_id')['total_price']
          .rolling(center=False, window=3, min_periods=1)
          .mean()
          .reset_index(drop=True)
)

result['price_avg'] = result.groupby('customer_id')['price_avg'].shift(periods=1)

result

/var/folders/04/6s8yy3js1wj1c5n_56_fvj680000gn/T/ipykernel_7217/1855451985.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)) \


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,97200.000000
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,58900.000000
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,50466.666667
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,82866.666667
...,...,...,...,...,...,...,...,...,...,...
4025,r4024,h_244,c_999,2016-10-06 18:01:34,2016-10-07,11:30:00,2016-10-09,3,59400,62400.000000
4026,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1,37200,60900.000000
4027,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,53000.000000
4028,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,37533.333333


In [9]:
import pandas.tseries.offsets as offsets
import operator

reserve_tb['reserve_datetime'] = pd.to_datetime(reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

sum_table = pd.merge(
    reserve_tb[['reserve_id', 'customer_id', 'reserve_datetime']],
    reserve_tb[['customer_id', 'reserve_datetime', 'total_price']].rename(columns={'reserve_datetime': 'reserve_datetime_before'}),
    on='customer_id'
)

sum_table = sum_table[operator.and_(
    sum_table['reserve_datetime'] > sum_table['reserve_datetime_before'],
    sum_table['reserve_datetime'] + offsets.Day(-90) <= sum_table['reserve_datetime_before']
)].groupby('reserve_id')['total_price'].sum().reset_index()

sum_table.columns = ['reserve_id', 'total_price_sum']

pd.merge(reserve_tb, sum_table, on='reserve_id', how='left').fillna(0)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,20600.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.0
...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,0.0
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,0.0
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,0.0
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,0.0


In [10]:
import datetime
from dateutil.relativedelta import relativedelta

month_mst = pd.DataFrame({
    'year_month': [(datetime.date(2017, 1, 1) + relativedelta(months=x)).strftime("%Y%m") for x in range(0, 3)]
})

customer_tb['join_key'] = 0
month_mst['join_key'] = 0

customer_mst = pd.merge(
    customer_tb[['customer_id', 'join_key']], month_mst, on='join_key'
)

customer_mst

,customer_id,join_key,year_month
0,c_1,0,201701
1,c_1,0,201702
2,c_1,0,201703
3,c_2,0,201701
4,c_2,0,201702
...,...,...,...
2995,c_999,0,201702
2996,c_999,0,201703
2997,c_1000,0,201701
2998,c_1000,0,201702
